In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from tabulate import tabulate
import plotly.graph_objects as go

#binary encoded chromosome
# Constants for bit lengths based on your setup
BITS_PER_COURSE = 4  # Allows for up to 16 courses
BITS_PER_ROOM = 3    # Allows for up to 8 rooms
BITS_PER_PROFESSOR = 3  # Allows for up to 8 professors
BITS_PER_TIMESLOT = 5  # Allows for up to 32 timeslots

def encode_gene(course_id, room_id, professor_id, timeslot_id):
    return (format(course_id, f'0{BITS_PER_COURSE}b') +
            format(room_id, f'0{BITS_PER_ROOM}b') +
            format(professor_id, f'0{BITS_PER_PROFESSOR}b') +
            format(timeslot_id, f'0{BITS_PER_TIMESLOT}b'))


def decode_gene(gene):
    course = int(gene[0:BITS_PER_COURSE], 2)
    room = int(gene[BITS_PER_COURSE:BITS_PER_COURSE+BITS_PER_ROOM], 2)
    professor = int(gene[BITS_PER_COURSE+BITS_PER_ROOM:BITS_PER_COURSE+BITS_PER_ROOM+BITS_PER_PROFESSOR], 2)
    timeslot = int(gene[-BITS_PER_TIMESLOT:], 2)
    return course, room, professor, timeslot


# Set the options to display all rows and columns
pd.set_option('display.max_rows', None)  # None means show all rows
pd.set_option('display.max_columns', None)  # None means show all columns
pd.set_option('display.width', None)  # None means use maximum possible width to display

afternoon_slots = [3, 4, 5]  # Indexes for afternoon slots

# Parameters
n_iter = 200
n_pop = 100
r_cross = 0.8  # Increased crossover rate
r_mut = 0.1  # mutation rate


# Problem specifics
n_courses = 10
n_professors = 7
n_timeslots = 20  # Example: 4 slots per day over 5 days

# Assuming 10 courses, some are labs, and have different section strengths
course_details = {
    i: {
        'lab': i % 3 == 0,  # Every third course is a lab
        'section_strength': 30 if i % 2 == 0 else 45  # Alternate high and low section strengths
    }
    for i in range(10)
}

# Room details
n_rooms = 5
room_capacities = [60, 60, 120, 120, 60]
room_floors = [1, 1, 2, 2, 1]  # Floor assignments for each room

# Example: Adjusting timeslot boundaries to include 15-minute breaks
timeslot_boundaries = [
    "8:30-9:50",   # 1 hour 20 minutes class + 15 minutes break
    "10:05-11:25", # Starts 15 minutes after the last class ends
    "11:40-1:00",  # and so on...
    "1:15-2:35",
    "2:50-4:10",
    "4:25-5:45"
]

def conflict_analysis(individual, course_details, room_capacities):
    professor_conflicts_count = 0
    room_conflicts_count = 0
    capacity_conflicts_count = 0
    adjacency_conflicts_count = 0

    professor_schedule = {}
    room_schedule = {}
    lecture_days = {}

    for course_id, room_id, professor_id, timeslot_id in individual:
        timeslot_day = timeslot_id // (n_timeslots // 5)  # Assuming 5 days, equally divided timeslots

        # Check professor double-booking
        if professor_id in professor_schedule and timeslot_id in professor_schedule[professor_id]:
            professor_conflicts_count += 1
        professor_schedule.setdefault(professor_id, set()).add(timeslot_id)

        # Check room double-booking and capacity
        if room_id in room_schedule and timeslot_id in room_schedule[room_id]:
            room_conflicts_count += 1
        if not check_room_capacity(room_id, course_details[course_id]['section_strength'], room_capacities):
            capacity_conflicts_count += 1
        room_schedule.setdefault(room_id, set()).add(timeslot_id)

        # Check for non-adjacent scheduling of courses
        lecture_days.setdefault(course_id, set()).add(timeslot_day)
        if len(lecture_days[course_id]) == 2 and abs(min(lecture_days[course_id]) - max(lecture_days[course_id])) == 1:
            adjacency_conflicts_count += 1

    return {
        "Total Conflicts": professor_conflicts_count + room_conflicts_count + capacity_conflicts_count + adjacency_conflicts_count,
        "Professor Conflicts": professor_conflicts_count,
        "Room Conflicts": room_conflicts_count,
        "Capacity Conflicts": capacity_conflicts_count,
        "Adjacency Conflicts": adjacency_conflicts_count
    }


def create_individual(course_details, n_rooms, n_professors, n_timeslots, room_capacities):
    individual = []
    morning_slots = [0, 1, 2]  # Indexes for morning slots with breaks considered
    afternoon_slots = [3, 4, 5]  # Indexes for afternoon slots
    for course_id in course_details:
        section_strength = course_details[course_id]['section_strength']
        suitable_rooms = [r for r in range(n_rooms) if room_capacities[r] >= section_strength]
        if course_details[course_id]['lab']:
            for day in range(5):  # Assume labs can be any day
                start_slot = day * 6 + afternoon_slots[0]  # Start at the first afternoon slot each day
                individual.append((course_id, np.random.choice(suitable_rooms), np.random.randint(0, n_professors), start_slot))
                individual.append((course_id, np.random.choice(suitable_rooms), np.random.randint(0, n_professors), start_slot + 1))
        else:
            for day in range(5):  # Schedule theory courses on non-adjacent days
                chosen_slot = np.random.choice(morning_slots)
                timeslot_id = day * 6 + chosen_slot
                individual.append((course_id, np.random.choice(suitable_rooms), np.random.randint(0, n_professors), timeslot_id))
    return individual



def check_room_capacity(room_id, section_strength, room_capacities):
    return section_strength <= room_capacities[room_id]

def objective(individual, course_details, room_capacities, room_floors):
    conflicts = 0
    room_usage = {}
    professor_schedule = {}
    section_room_week = {}
    courses_per_professor = {}
    courses_per_section = {}
    lecture_days = {course_id: [] for course_id in course_details}

    for course_id, room_id, professor_id, timeslot_id in individual:
        section_strength = course_details[course_id]['section_strength']

        # Check for room and professor double booking within the same timeslot
        if timeslot_id not in room_usage:
            room_usage[timeslot_id] = set()
        if timeslot_id not in professor_schedule:
            professor_schedule[timeslot_id] = set()

        if room_id in room_usage[timeslot_id]:
            conflicts += 1  # Room is double-booked
        if professor_id in professor_schedule[timeslot_id]:
            conflicts += 1  # Professor is double-booked
        if not check_room_capacity(room_id, section_strength, room_capacities):
            conflicts += 1  # Room capacity is exceeded

        room_usage[timeslot_id].add(room_id)
        professor_schedule[timeslot_id].add(professor_id)

        # Track lectures per week for each course
        day_index = timeslot_id // 6  # Assuming 6 timeslots per day adjusted for breaks
        lecture_days[course_id].append(day_index)

        # Section-room consistency within a week
        section_key = (course_id // len(course_details), day_index)  # Course divided by total to get section ID
        if section_key in section_room_week:
            if section_room_week[section_key] != room_id:
                conflicts += 1  # Different rooms in the same week for the same course-section
        else:
            section_room_week[section_key] = room_id

        # Tracking maximum courses per professor and per section
        courses_per_professor[professor_id] = courses_per_professor.get(professor_id, 0) + 1
        courses_per_section[course_id // len(course_details)] = courses_per_section.get(course_id // len(course_details), 0) + 1

    # Enforcing limits on courses per professor and per section
    for count in courses_per_professor.values():
        if count > 3:
            conflicts += (count - 3)
    for count in courses_per_section.values():
        if count > 5:
            conflicts += (count - 5)

    # Ensuring courses are not on adjacent days unless it's a lab needing consecutive slots
    for course_id, days in lecture_days.items():
        if course_details[course_id]['lab']:
            if len(set(days)) != 1:
                conflicts += 1  # Labs should be on a single day in consecutive slots
        else:
            if len(set(days)) < 2 or min(abs(d1 - d2) for d1 in days for d2 in days if d1 != d2) == 1:
                conflicts += 1  # Non-lab courses should not be on adjacent days

    return -conflicts  # Negative score for minimization, as lower conflicts are better

def objective_with_soft_constraints(individual, course_details, room_capacities, room_floors):
    hard_score = objective(individual, course_details, room_capacities, room_floors)
    soft_penalty = 0

    # Existing soft constraints
    room_assignments = {}
    professor_times = {}
    professor_last_floor = {}  # Track the last floor each professor was on

    for course_id, room_id, professor_id, timeslot_id in individual:
        day_part = 'morning' if timeslot_id % (n_timeslots // 5) < 10 else 'afternoon'
        if day_part not in professor_times.get(professor_id, {}):
            professor_times.setdefault(professor_id, {})[day_part] = []
        professor_times[professor_id][day_part].append(timeslot_id)

        # Minimizing room changes
        if course_id not in room_assignments:
            room_assignments[course_id] = room_id
        elif room_assignments[course_id] != room_id:
            soft_penalty += 1  # Penalize room changes across different days

        # Floor change penalty
        if professor_id in professor_last_floor:
            if professor_last_floor[professor_id] != room_floors[room_id]:
                soft_penalty += 1  # Add penalty for changing floors
        professor_last_floor[professor_id] = room_floors[room_id]

    # Check for consecutive blocks for each professor
    for timeslots in professor_times.values():
        for part, times in timeslots.items():
            if len(times) > 1:
                sorted_times = sorted(times)
                for i in range(len(sorted_times) - 1):
                    if sorted_times[i+1] != sorted_times[i] + 1:
                        soft_penalty += 1  # Penalize non-consecutive blocks

    # Balance morning and afternoon classes
    morning_count = sum(len(times.get('morning', [])) for times in professor_times.values())
    afternoon_count = sum(len(times.get('afternoon', [])) for times in professor_times.values())
    if abs(morning_count - afternoon_count) > 5:
        soft_penalty += abs(morning_count - afternoon_count) - 5

    return hard_score - soft_penalty

def local_search(best, objective_function, course_details, room_capacities, room_floors):
    best_eval = objective_function(best, course_details, room_capacities, room_floors)
    for _ in range(100):  # Number of local search iterations
        candidate = best.copy()
        idx = np.random.randint(len(candidate))
        new_course_id = np.random.randint(0, len(course_details))
        new_room_id = np.random.choice([r for r in range(len(room_capacities)) if room_capacities[r] >= course_details[new_course_id]['section_strength']])
        new_professor_id = np.random.randint(0, n_professors)
        new_timeslot_id = np.random.randint(0, n_timeslots)
        candidate[idx] = (new_course_id, new_room_id, new_professor_id, new_timeslot_id)
        candidate_eval = objective_function(candidate, course_details, room_capacities, room_floors)
        if candidate_eval > best_eval:  # Since you're minimizing, check if new evaluation is better (less negative)
            best, best_eval = candidate, candidate_eval
    return best


def enhanced_visualization(solution):
    import pandas as pd
    df = pd.DataFrame(solution, columns=['Course', 'Room', 'Professor', 'Timeslot'])
    print(pd.crosstab(df['Timeslot'], [df['Course'], df['Room']], dropna=False).fillna('---'))


def selection(pop, scores):
    selection_ix = np.random.randint(len(pop))
    for _ in range(2):
        ix = np.random.randint(len(pop))
        if scores[ix] < scores[selection_ix]:
            selection_ix = ix
    return pop[selection_ix]


def is_valid(individual, course_details, room_capacities):
    # Initialize data structures to track room and professor assignments
    room_usage = {}
    professor_usage = {}
    room_capacity_violations = 0

    for course_id, room_id, professor_id, timeslot_id in individual:
        # Ensure the room capacity is not exceeded
        if room_capacities[room_id] < course_details[course_id]['section_strength']:
            room_capacity_violations += 1
            return False  # Return immediately if room is too small for the section

        # Track usage of rooms and professors per timeslot to detect conflicts
        if timeslot_id not in room_usage:
            room_usage[timeslot_id] = set()
        if timeslot_id not in professor_usage:
            professor_usage[timeslot_id] = set()

        # Check for room conflicts (room double-booking)
        if room_id in room_usage[timeslot_id]:
            return False  # Invalid if the same room is booked twice in the same timeslot
        room_usage[timeslot_id].add(room_id)

        # Check for professor conflicts (professor double-booking)
        if professor_id in professor_usage[timeslot_id]:
            return False  # Invalid if the same professor is booked twice in the same timeslot
        professor_usage[timeslot_id].add(professor_id)

    # Check for additional constraints as needed, e.g., specific scheduling requirements for labs/theory
    # This would involve more detailed checking based on your course_details structure

    # If all checks pass, the schedule is considered valid
    return True

def crossover(p1, p2, r_cross, is_valid, course_details, room_capacities):
    if np.random.rand() < r_cross:
        pt = np.random.randint(1, len(p1)-2)  # Ensure crossover point is not at the very ends
        c1, c2 = p1[:pt] + p2[pt:], p2[:pt] + p1[pt:]
        if not (is_valid(c1, course_details, room_capacities) and is_valid(c2, course_details, room_capacities)):
            return [p1, p2]  # Revert if invalid
        return [c1, c2]
    return [p1.copy(), p2.copy()]



def genetic_algorithm(create_individual, objective_with_soft_constraints, course_details, room_capacities, room_floors, n_iter, n_pop, r_cross, r_mut, local_search_frequency=20):
    # Initial population
    pop = [create_individual(course_details, n_rooms, n_professors, n_timeslots, room_capacities) for _ in range(n_pop)]
    best, best_eval = pop[0], objective_with_soft_constraints(pop[0], course_details, room_capacities, room_floors)

    # Main loop
    for gen in range(n_iter):
        # Evaluate all candidates in the population
        scores = [objective_with_soft_constraints(ind, course_details, room_capacities, room_floors) for ind in pop]

        # Check for new best solution
        for i in range(n_pop):
            if scores[i] < best_eval:  # Remember we are minimizing conflicts
                best, best_eval = pop[i], scores[i]
                print(f"> Generation {gen}, new best score = {best_eval}")

        # Tournament selection
        selected = [selection(pop, scores) for _ in range(n_pop)]

        # Crossover and mutation operations
        children = []
        for i in range(0, n_pop, 2):
            p1, p2 = selected[i], selected[i+1]
            for c in crossover(p1, p2, r_cross, is_valid, course_details, room_capacities):
                mutation(c, r_mut, course_details, n_rooms, n_professors, n_timeslots, room_capacities)  # Added room_capacities here
                if gen % local_search_frequency == 0:
                    c = local_search(c, objective_with_soft_constraints, course_details, room_capacities, room_floors)
                children.append(c)

        pop = children

    # Post-optimization local search on the best result
    best = local_search(best, objective_with_soft_constraints, course_details, room_capacities, room_floors)
    best_eval = objective_with_soft_constraints(best, course_details, room_capacities, room_floors)

    return best, best_eval



    # Post-optimization local search on the best result
    best = local_search(best, objective_with_soft_constraints, course_details, room_capacities, room_floors)
    best_eval = objective_with_soft_constraints(best, course_details, room_capacities, room_floors)

    return best, best_eval


def mutation(individual, r_mut, course_details, n_rooms, n_professors, n_timeslots, room_capacities):
    for i in range(len(individual)):
        if np.random.rand() < r_mut:
            original = individual[i]
            course_id = original[0]  # Maintain course ID
            new_room_id = np.random.choice([r for r in range(n_rooms) if room_capacities[r] >= course_details[course_id]['section_strength']])
            new_professor_id = np.random.randint(0, n_professors)
            new_timeslot = np.random.randint(0, n_timeslots)

            candidate = (course_id, new_room_id, new_professor_id, new_timeslot)
            # Set new values conditionally, validating the entire individual if necessary
            individual[i] = candidate
            if not is_valid(individual, course_details, room_capacities):  # Include the necessary arguments here
                individual[i] = original  # Revert if the mutation creates an invalid schedule

def run_complete_algorithm():
    # Running the genetic algorithm with local search
    best_solution, best_evaluation = genetic_algorithm(
        create_individual,  # Create individual function
        objective_with_soft_constraints,  # Objective function
        course_details,  # Course details
        room_capacities,  # Room capacities
        room_floors,  # Room floors
        n_iter,  # Number of iterations
        n_pop,  # Population size
        r_cross,  # Crossover rate
        r_mut,  # Mutation rate
        local_search_frequency=20  # Apply local search every 20 generations
    )
    print("Best Evaluation:", best_evaluation)

    enhanced_visualization(best_solution)
    simulate_timetable()

    plotly_timetable_visualization(best_solution, course_details)
    #export_timetable_to_excel(best_solution, course_details)

def simulate_timetable():
    # Assuming room details and course details are accessible globally
    days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]
    times = ["8:30-9:50", "10:05-11:25", "11:40-1:00", "1:15-2:35", "2:50-4:10", "4:25-5:45"]
    # Adding room details dynamically
    rooms = [f"Room {i+1} (Floor {room_floors[i]}, Cap {room_capacities[i]})" for i in range(5)]
    courses = ["Math", "Physics", "Physics Lab", "History", "Engineering", "Computer Science", "Biology", "Chemistry Lab", "Literature", "Economics"]
    professors = ["Prof. Smith", "Prof. Johnson", "Prof. Williams", "Prof. Jones", "Prof. Brown", "Prof. Davis", "Prof. Miller"]

    # Creating a DataFrame to simulate the timetable
    timetable = pd.DataFrame(index=times, columns=rooms)
    for day in days:
        for room in rooms:
            for time in times:
                if np.random.rand() > 0.7:  # Randomly decide if the slot should be filled
                    course = np.random.choice(courses)
                    professor = np.random.choice(professors)
                    # Parsing room details for capacity and floor
                    room_capacity = int(room.split('Cap ')[-1].rstrip(')'))
                    timetable.at[time, room] = f"{course} ({professor}, {room_capacity} students)"
                else:
                    timetable.at[time, room] = "Empty"

        # Output each day's timetable to a separate sheet in the same Excel file
        with pd.ExcelWriter('Timetable.xlsx', mode='a', engine='openpyxl', if_sheet_exists='replace') as writer:
            timetable.to_excel(writer, sheet_name=day)

def plotly_timetable_visualization(solution, course_details):
    # Course, Room, Professor, and Timeslot labels
    course_names = {
        0: "Mathematics",
        1: "Physics Lab",
        2: "History",
        3: "Engineering",
        4: "Computer Science Lab",
        5: "Biology",
        6: "Chemistry Lab",
        7: "Chemistry",
        8: "Literature",
        9: "Economics"
    }
    class_and_section = {
        0: ("Class A", "Section 1"),
        1: ("Class B", "Section 2"),
        2: ("Class C", "Section 3"),
        3: ("Class D", "Section 1"),
        4: ("Class E", "Section 2"),
        5: ("Class F", "Section 3"),
        6: ("Class G", "Section 1"),
        7: ("Class H", "Section 2"),
        8: ("Class I", "Section 3"),
        9: ("Class J", "Section 1")
    }
    day_names = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]
    timeslot_boundaries = ["8:30-9:50", "10:00-11:20", "11:30-12:50", "1:00-2:20", "2:30-3:50", "4:00-5:20"]

    # DataFrame creation and manipulation
    df = pd.DataFrame(solution, columns=['Course', 'Room', 'Professor', 'Timeslot'])
    df['Course Name'] = df['Course'].map(course_names)
    df['Class'] = df['Course'].apply(lambda x: class_and_section[x][0])
    df['Section'] = df['Course'].apply(lambda x: class_and_section[x][1])
    df['Course Type'] = df['Course'].apply(lambda x: 'Lab' if course_details[x]['lab'] else 'Theory')
    df['Detailed Course Name'] = df.apply(lambda x: f"{x['Course Name']} ({x['Course Type']}) - {x['Class']} {x['Section']}", axis=1)
    df['Day'] = df['Timeslot'] // 6
    df['Day Name'] = df['Day'].apply(lambda x: day_names[x])
    df['Timeslot Name'] = df['Timeslot'].apply(lambda x: timeslot_boundaries[x % 6])
    df['Room Name'] = df['Room'].apply(lambda x: f"Room {x+1}")
    df['Professor Name'] = df['Professor'].apply(lambda x: f"Prof {x+1}")

    # Organize data for Plotly with sorting
    for day in day_names:
        day_df = df[df['Day Name'] == day]
        day_df = day_df.sort_values(by='Timeslot')  # Sorting by timeslot within the day
        fig = go.Figure(data=[go.Table(
            header=dict(values=['Timeslot', 'Detailed Course Name', 'Room Name', 'Professor Name'], fill_color='paleturquoise', align='left'),
            cells=dict(values=[
                day_df['Timeslot Name'], day_df['Detailed Course Name'], day_df['Room Name'], day_df['Professor Name']
            ], fill_color='lavender', align='left'))
        ])
        fig.update_layout(title=f'Timetable for {day}', width=800, height=600)
        fig.show()



run_complete_algorithm()



> Generation 0, new best score = -394
> Generation 0, new best score = -406
> Generation 0, new best score = -409
> Generation 0, new best score = -414
> Generation 0, new best score = -415
Best Evaluation: -351
Course    0              1              2              3              4              5           \
Room      0  1  2  3  4  0  1  2  3  4  0  1  2  3  4  0  1  2  3  4  0  1  2  3  4  0  1  2  3   
Timeslot                                                                                          
0         1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  1   
1         0  0  0  0  0  1  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  1  0  0  0  0  0  0  0   
2         0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0   
3         0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0   
4         1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0